In [ ]:
## dont run beyond 60 lack of computation - 92% ram 
## try 40 next

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import glob
import pathlib
import PIL
import os
import tarfile
import cv2
import numpy as np
import time

lower_age_limit = 1
upper_age_limit = 5

In [ ]:
tar_files = ['part1.tar.gz', 'part2.tar.gz', 'part3.tar.gz']
output_folder = os.path.join(os.getcwd(), 'all_images')

if not os.path.exists(output_folder):
    os.makedirs(output_folder)

    for tar_file in tar_files:
        with tarfile.open(tar_file, 'r:gz') as tar:
            for member in tar.getmembers():
                # Remove the leading directory name from the member's name
                member.name = os.path.basename(member.name)
                tar.extract(member, output_folder)

In [ ]:
data_dir = pathlib.Path('.//all_images')

In [ ]:
image_count = len(list(data_dir.glob('**/*.jpg')))
print(image_count)

In [ ]:
# age_1 = list(data_dir.glob('1_*'))
# PIL.Image.open(str(age_1[1]))

In [ ]:
age_images_dict = {}
ages_labels_dict = {}

for i in range(lower_age_limit, upper_age_limit):
    age_images_dict[i] = list(data_dir.glob(f'{i}_*'))
    ages_labels_dict[i] = i-1

# some ages are missing 

In [ ]:
# img = cv2.imread(str(age_images_dict[3][0]))
# img.shape

for x in age_images_dict.values():
    print(len(x))

In [ ]:
# start = time.time()
# x, y = [], []

# for age_number, images in age_images_dict.items():
#     for image in images:
#         try:
#             img = cv2.imread(str(image))
#             resized_img = cv2.resize(img,(180,180))
#         except cv2.error as e:
#             print(f"Error processing {image}: {str(e)}")
#             continue
#         x.append(resized_img)
#         y.append(ages_labels_dict[age_number])
        
# x = np.array(x)/255
# y = np.array(y)
# print(time.time()-start)

In [ ]:
from concurrent.futures import ThreadPoolExecutor, as_completed
def process_image(image):
    try:
        img = cv2.imread(str(image))
        resized_img = cv2.resize(img,(180,180))
        return resized_img
    except cv2.error as e:
        print(f"Error processing {image}: {str(e)}")
        return None

x, y = [], []

with ThreadPoolExecutor() as executor:
    futures = []
    for age_number, images in age_images_dict.items():
        for image in images:
            future = executor.submit(process_image, image)
            futures.append((future, age_number))

    for future, age_number in futures:
        result = future.result()
        if result is not None:
            x.append(result)
            y.append(ages_labels_dict[age_number])

x = np.array(x)
y = np.array(y)

In [ ]:
np.savez( os.getcwd() + '/ age_image_data.npz', x, y)
del x, y, age_images_dict, ages_labels_dict, futures, result, images

# # run from here if saved .npz file

In [1]:
import numpy as np
import tensorflow as tf
import os

from tensorflow.keras import datasets, layers, models
from tensorflow import keras
from tensorflow.keras.models import Sequential
from sklearn.utils import class_weight
from imblearn.over_sampling import SMOTE
from keras.callbacks import EarlyStopping

In [2]:
# def macro_f1(y_true, y_pred):
#     y_pred = tf.argmax(y_pred, axis=-1)
#     return f1_score(y_true, y_pred, average='macro')
from tensorflow.keras.metrics import Metric

class MacroF1(Metric):
    def __init__(self, name='macro_f1', **kwargs):
        super().__init__(name=name, **kwargs)
        self.true_positives = self.add_weight(name='tp', initializer='zeros')
        self.false_positives = self.add_weight(name='fp', initializer='zeros')
        self.false_negatives = self.add_weight(name='fn', initializer='zeros')

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_pred = tf.argmax(y_pred, axis=-1)
        y_true = tf.cast(y_true, tf.int32)
        y_pred = tf.cast(y_pred, tf.int32)

        true_positives = tf.cast(tf.math.count_nonzero(y_true * y_pred, axis=0), tf.float32)
        false_positives = tf.cast(tf.math.count_nonzero((1 - y_true) * y_pred, axis=0), tf.float32)
        false_negatives = tf.cast(tf.math.count_nonzero(y_true * (1 - y_pred), axis=0), tf.float32)

        self.true_positives.assign_add(tf.reduce_sum(true_positives))
        self.false_positives.assign_add(tf.reduce_sum(false_positives))
        self.false_negatives.assign_add(tf.reduce_sum(false_negatives))

    def result(self):
        precision = self.true_positives / (self.true_positives + self.false_positives + 1e-6)
        recall = self.true_positives / (self.true_positives + self.false_negatives + 1e-6)
        f1 = 2 * precision * recall / (precision + recall + 1e-6)
        macro_f1 = tf.reduce_mean(f1)
        return macro_f1

    def reset_state(self):
        self.true_positives.assign(0)
        self.false_positives.assign(0)
        self.false_negatives.assign(0)

In [3]:
with np.load(os.getcwd() + '/ age_image_data.npz', mmap_mode='c') as data:
    x = data['arr_0']
    y = data['arr_1']

num_classes = len(np.unique(y))

In [4]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, y, random_state=0)
del x, y
class_weights = dict(zip(np.unique(y_train), class_weight.compute_class_weight('balanced',
                                classes = np.unique(y_train), y = y_train)))

sm = SMOTE(random_state=42)
X_train_reshaped = X_train.reshape(X_train.shape[0], -1)
X_train_smote, y_train_smote = sm.fit_resample(X_train_reshaped, y_train)
X_train_smote = X_train_smote.reshape(X_train_smote.shape[0], 180, 180, 3) 

del X_train, y_train

In [5]:
data_augmentation = keras.Sequential(
    [
        tf.keras.layers.Rescaling(scale = 1./255, offset=0.0),
        layers.experimental.preprocessing.RandomFlip("horizontal", input_shape=(180, 180, 3)),
        layers.experimental.preprocessing.RandomRotation(0.1),
        layers.experimental.preprocessing.RandomZoom(0.1)    
    ]
)

model = Sequential([
    
    data_augmentation,
    
    layers.Conv2D(64, 3, padding='same', activation='relu', input_shape = (180,180,3)),
    layers.MaxPooling2D(),
    
    layers.Conv2D(64, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    
    layers.Conv2D(64, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    
    layers.Conv2D(64, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(num_classes, activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=[MacroF1()])
es = EarlyStopping(monitor='macro_f1', verbose=0)

In [6]:
model.fit(X_train_smote, y_train_smote, epochs=10, class_weight=class_weights, callbacks=[es])
print(model.evaluate(X_train_smote,y_train_smote)[1])
del X_train_smote, y_train_smote

Epoch 1/10
120/120 [==============================] - 126s 1s/step - loss: 1.7495 - macro_f1: 0.4998
Epoch 2/10
120/120 [==============================] - 24s 202ms/step - loss: 1.3978 - macro_f1: 0.5000
0.4999995231628418


In [7]:
X_test = X_test.reshape(X_test.shape[0], 180, 180, 3)
print(model.evaluate(X_test,y_test)[1])
del X_test,y_test

19/19 [==============================] - 4s 200ms/step - loss: 1.6658 - macro_f1: 0.4283
0.42834967374801636
